In [ ]:
import os
import pandas as pd
import sys

# project_path = "."
project_path = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(project_path)
import BraiAn

In [ ]:
EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "IEGs/cFos_Arc1", True
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "rebased_on_mjd", False
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "ieg", False
# EXPERIMENT_DIRECTORY, USE_REMOTE_DATA = "experiment", True

# ###################################### REMOTE DIRECTORIES #####################################
IS_COLLABORATION_PROJ = False
# COLLABORATION_DIRECTORY = "Mathias Schmidt/soumnya"
COLLABORATION_DIRECTORY = "Mathias Schmidt/shila"

if USE_REMOTE_DATA:
    DATA_ROOT, PLOTS_ROOT = BraiAn.remote_dirs(EXPERIMENT_DIRECTORY, IS_COLLABORATION_PROJ, COLLABORATION_DIRECTORY)
else:
    DATA_ROOT = f"../data/experiments/{EXPERIMENT_DIRECTORY}"
data_input_path = os.path.join(DATA_ROOT, "BraiAn_output")

In [ ]:
metric, kwargs, max_value = BraiAn.BrainMetrics.DENSITY,           dict(),                                 None
# metric, kwargs, max_value = BraiAn.BrainMetrics.SIMILARITY_INDEX,  dict(marker1="cFos", marker2="Arc1"),   1
# metric, kwargs, max_value = BraiAn.BrainMetrics.OVERLAPPING,       dict(marker1="cFos", marker2="Arc1"),   1

In [ ]:
# from https://help.brain-map.org/display/api/Downloading+an+Ontology%27s+Structure+Graph
# StructureGraph id=1
path_to_allen_json = os.path.join(project_path, "data", "AllenMouseBrainOntology.json")
BraiAn.cache(path_to_allen_json, "http://api.brain-map.org/api/v2/structure_graph_download/1.json")
brain_onthology = BraiAn.AllenBrainHierarchy(path_to_allen_json,
                    ["fiber tracts", "VS", "grv", "retina", "CB"], # "FRP", "VISpl1"],
                    version="ccfv3")

In [ ]:
REGIONS_TO_PLOT_SELECTION_METHOD, HEIGHT = "structural level 8", 5000
# REGIONS_TO_PLOT_SELECTION_METHOD, HEIGHT = "major divisions", 1000
# REGIONS_TO_PLOT_SELECTION_METHOD, HEIGHT = "smallest", 15000
match REGIONS_TO_PLOT_SELECTION_METHOD:
    case "summary structures":
        # selects the Summary Strucutures
        path_to_summary_structures = os.path.join(project_path, "data", "AllenSummaryStructures.csv")
        brain_onthology.select_from_csv(path_to_summary_structures)
    case "major divisions":
        brain_onthology.select_regions(BraiAn.MAJOR_DIVISIONS)
    case "smallest":
        brain_onthology.select_leaves()
    case s if s.startswith("depth"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            depth = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'depth' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        brain_onthology.select_at_depth(depth)
    case s if s.startswith("structural level"):
        n = REGIONS_TO_PLOT_SELECTION_METHOD.split(" ")[-1]
        try:
            level = int(n)
        except Exception:
            raise Exception("Could not retrieve the <n> parameter of the 'structural level' method for 'REGIONS_TO_PLOT_SELECTION_METHOD'")
        brain_onthology.select_at_structural_level(level)
    case _:
        raise Exception(f"Invalid value '{REGIONS_TO_PLOT_SELECTION_METHOD}' for REGIONS_TO_PLOT_SELECTION_METHOD")
selected_regions = brain_onthology.get_selected_regions()
print(f"You selected {len(selected_regions)} regions to plot.")

In [ ]:
# group_1: BraiAn.AnimalGroup = BraiAn.AnimalGroup.from_csv("CTX", data_input_path, f"cell_counts_CTX_density.csv")
# group_2: BraiAn.AnimalGroup = BraiAn.AnimalGroup.from_csv("FC", data_input_path, f"cell_counts_FC_density.csv")
group_1 = BraiAn.AnimalGroup(
            "HC",
            [BraiAn.AnimalBrain.from_csv(a, data_input_path, "sum") for a in ["287HC", "345HC", "346HC", "371HC"]],
            metric=metric,
            merge_hemispheres=True,
            brain_onthology=brain_onthology,
            fill_nan=True,
            **kwargs
        )
group_2 = BraiAn.AnimalGroup(
            "CTX",
            [BraiAn.AnimalBrain.from_csv(a, data_input_path, "sum") for a in ["329CTX", "331CTX", "355CTX", "400CTX", "401CTX", "402CTX"]],
            metric=metric,
            merge_hemispheres=True,
            brain_onthology=brain_onthology,
            fill_nan=True,
            **kwargs
        )
group_3 = BraiAn.AnimalGroup(
            "FC",
            [BraiAn.AnimalBrain.from_csv(a, data_input_path, "sum") for a in ["367FC", "368FC", "369FC", "426FC", "427FC", "428FC"]],
            metric=metric,
            merge_hemispheres=True,
            brain_onthology=brain_onthology,
            fill_nan=True,
            **kwargs
        )

if not group_1.is_comparable(group_2):
    raise ImportError("Group 1 and Group 2 are not comparable!\n\
Please check that you're reading two groups that normalized on the same brain regions and on the same marker")

In [ ]:
pls = {
    marker: group_2.pls_regions(group_3, selected_regions, n_permutations=5000, n_bootstrap=5000, marker=marker)
    for marker in group_2.markers
}
pls

In [ ]:
for marker, scores in pls.items():
    plot_pls_threshold = BraiAn.PLS.norm_threshold(0.05, two_tailed=True)
    salient_regions = scores.data[scores.data.abs() > plot_pls_threshold]
    f = BraiAn.plot_salient_regions(salient_regions, brain_onthology,
                                    title=f"{group_2.name} vs {group_3.name} - {marker} {str(group_2.metric)}<br>salient regions",
                                    thresholds=[BraiAn.PLS.norm_threshold(p, two_tailed=True) for p in (0.05, 0.01)],
                                    title_size=30, axis_size=22,
                                    use_acronyms=True, use_acronyms_in_mjd=False,
                                    mjd_opacity=0.3,
                                    width=1000, barheight=30)
    # f.write_html(
    #     f"salience_{group_2.name}vs{group_3.name}_{str(group_2.metric)}_{marker}_{REGIONS_TO_PLOT_SELECTION_METHOD}"
    #         .lower()
    #         .replace(".", "_")
    #         .replace(" ", "_")+".html",
    #     config=dict(toImageButtonOptions=dict(format="svg")))
    f.show()

In [ ]:
pls_threshold = BraiAn.PLS.norm_threshold(p=0.05, two_tailed=True)
f = BraiAn.plot_gridgroups(
                       # [group_2, group_3],
                       [group_1, group_2, group_3],
                       selected_regions,
                       # list(set(selected_regions)),
                       # "cFos+Arc1",
                       *group_1.markers[:2], #"cFos", "Arc1",
                       brain_onthology=brain_onthology,
                       # brain_onthology=None,
                       plot_scatter=True, #False,
                       markers_salience_scores=pls,
                       pls_n_permutations=5000, pls_n_bootstrap=5000,
                       pls_threshold=pls_threshold,
                       height=HEIGHT, width=None,
                       barplot_width=0.7, space_between_markers=0.02,
                       groups_marker1_colours=["LightCoral", "SandyBrown", "green"],
                       groups_marker2_colours=["IndianRed", "Orange", "lightgreen"],
                       color_heatmap="deep_r", max_value=max_value)
# f.write_html(
#     f"gridplot_{str(group_1.metric)}_{REGIONS_TO_PLOT_SELECTION_METHOD.lower()}_pls_{pls_threshold:0.2f}"
#         .replace(".", "_")
#         .replace(" ", "_")+".html",
#     config=dict(toImageButtonOptions=dict(format="svg")))
f.show()